# 03 - Interactive Viz

## Important
In order to see the rendered maps, you can follow this [link](https://nbviewer.jupyter.org/github/armand33/applied_data_analysis_2017/blob/master/03%20-%20Interactive%20Viz/Homework%203.ipynb?flush_cache=true) to an online notebook viewer.

If link is dead, and to make sure you read the latest version of render, you can also go on the [notebook viewer website](https://nbviewer.jupyter.org/) and paste the github [link](https://github.com/armand33/applied_data_analysis_2017/blob/master/03%20-%20Interactive%20Viz/Homework%203.ipynb) of our notebook.


In [1]:
import pandas as pd
import numpy as np
import branca.colormap as cm
import folium
import json
import geopandas as gpd
import codecs

from geopandas import GeoSeries, GeoDataFrame
from folium.plugins import MarkerCluster

### General choices:

In all this homework, we chose the rates over the last quarter (for europe) and last month (for Switzerland).
We could have averaged over the last calendar year (that would have canceled some eventual season effects) but it would also have taken into account outdated data. **We chose to work on the latest data available.**

# 1) European unemployment

Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

#### Paths

In [2]:
europe_csv = 'data/lfsq_urgan_1_Data.csv'
topo_path = r'topojson/europe.topojson.json'

In [3]:
topo_data = json.load(open(topo_path))

#### Extracting unemployment rates

We use the unemployement rates of the second quarter of 2017. These are the most recent rates found on the eurostat website.

In [4]:
df = pd.read_csv(europe_csv)
df = df.loc[df.TIME == '2017Q2'].loc[df.SEX == 'Total'].loc[:, ['GEO', 'Value']]
df.columns = ['country', 'rate']
df = df[6:].reset_index(drop=True)
df.head()

,country,rate
0,Belgium,7.0
1,Bulgaria,6.3
2,Czech Republic,3.0
3,Denmark,5.5
4,Germany (until 1990 former territory of the FRG),3.8


Changing some names.

In [5]:
df.loc[df.country == 'Germany (until 1990 former territory of the FRG)', 'country'] = 'Germany'
df.loc[df.country == 'Former Yugoslav Republic of Macedonia, the', 'country'] = 'The former Yugoslav Republic of Macedonia'

#### Retrieving the two-letter codes for each country. 

These codes are contained in the topojson file provided. They will be useful in order to build the map.

In [6]:
countries = {}
for d in topo_data['objects']['europe']['geometries']:
    countries[d['properties']['NAME']] = d['id']

#### Enterring those codes in the dataframe

In [7]:
for i in df.index:
    df.loc[i, 'code'] = countries[df.loc[i, 'country']]

In [8]:
df = df.sort_values('rate').reset_index(drop=True)
df.head()

,country,rate,code
0,Czech Republic,3.0,CZ
1,Iceland,3.4,IS
2,Germany,3.8,DE
3,Malta,4.1,MT
4,United Kingdom,4.3,GB


#### Choosing intervals for colors :

We use 5 classes:
* 1 for outliers (Spaine, Greece and Macedonia)
* 4 for the other countries using quantile classification 

In [9]:
df.loc[:len(df)-3, 'class'] = pd.qcut(df.rate[:len(df)-3], 4, labels = [i for i in range(1,5)])
df.loc[len(df)-3:, 'class'] = 5

Here is the number of countries for which the unemployment rate is in each interval.

In [10]:
df['class'].value_counts()

4.0    8
1.0    8
3.0    7
2.0    7
5.0    3
Name: class, dtype: int64

And here are the bounds for the intervals

In [11]:
t = [min(df['rate'])]
for i in range(1,6):
    tmp = df.loc[df['class'] == i]['rate'].values
    t.append(max(tmp))
    
t

[3.0,
 4.4000000000000004,
 6.2999999999999998,
 8.0999999999999996,
 11.0,
 22.600000000000001]

#### Building the map

In [12]:
m1 = folium.Map(location=[50, 10], zoom_start=4, tiles='cartodbpositron')

#### Data missing from the data set.
In order to show countries for which there is no data, we want to color them in black.

`present` is the set of countries for which we have the data

In [13]:
present = set(df['country'].values)

Then we extract two topo json files from the original ones. One for the missing data and one for the present one. It allows us to **apply different styles** to those countries.

In [14]:
missing_topo_data = json.load(open(topo_path))
l = missing_topo_data['objects']['europe']['geometries']
missing_topo_data['objects']['europe']['geometries'] = [d for d in l if d['properties']['NAME'] not in present]

In [15]:
present_topo_data = json.load(open(topo_path))
l = present_topo_data['objects']['europe']['geometries']
present_topo_data['objects']['europe']['geometries'] = [d for d in l if d['properties']['NAME'] in present]

#### Coloring missing countries in black.
Now we add a layer to the map for missing data and color it in black.

In [16]:
folium.TopoJson(missing_topo_data, object_path='objects.europe', style_function=lambda feature: {
    'color' : 'black',
    'opacity' : 0,
    'weight' : 2,
    'dashArray' : '5, 5'
        }).add_to(m1)

#### Chloropleth

We add a cholorpleth for the present data.

In [17]:
m1.choropleth(geo_data=present_topo_data, data=df,
             columns=['code', 'rate'], topojson='objects.europe',
             key_on='feature.id',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Unemployment rate (%)', threshold_scale=t, name='unemployment layer')

In [18]:
m1

#### Bringing interactivity

We want to add some markers on each country in order to get the exact value of unemployement rate by clicking on it.
To do that we use a data set called `country_centroids_primary` that provides the coordinates of centroids of countries. This data set comes from [Gotos]('http://gothos.info/resources/').

In [19]:
centro = pd.read_csv('data/country_centroids_primary.csv', sep=('\t')).loc[:, ['LAT', 'LONG', 'SHORT_NAME']]
centro.columns = ['lat', 'long', 'name']
centro.head()

,lat,long,name
0,33.000000,66.0,Afghanistan
1,41.000000,20.0,Albania
2,28.000000,3.0,Algeria
3,-14.333333,-170.0,American Samoa
4,42.500000,1.5,Andorra


Now for each country of europe, we add the marker on the centroid. By clicking on the marker, one get the required value.

In [20]:
m_c = MarkerCluster().add_to(m1)

for i in df.index:
    n = df.loc[i, 'country']
    p = df.loc[i, 'rate']
    if n == 'The former Yugoslav Republic of Macedonia':
        n = 'Macedonia'
    long, lat = centro.loc[centro.name == n, 'long'].values[0], centro.loc[centro.name == n, 'lat'].values[0]
    folium.Marker([lat, long], popup='{} : {}%'.format(n, p) , icon=folium.Icon(color='green')).add_to(m_c)

In [21]:
m1.save('maps/m1.html')
m1

This is the final map. Black countries are countries for which there is no data.

#### Comparing to the rate in Switzerland

We choose to plot countries that have lower unemployement rates in purple to blue and others on the same color scale as before (Yellow-Orange-Red).

In [22]:
df['diff'] = df['rate'] - df.loc[df.country == 'Switzerland', 'rate'].values
df['diff'] = df['diff'].astype(float).round(1)

Set of countries better and worse than Switzerland in term of unemployement.

In [23]:
better = set(df.loc[df['diff'] < 0, 'country'].values)
worse = set(df.loc[df['diff'] > 0, 'country'].values)

Extracting corresponding topo_json files in order to build independant cholorpleth.

In [24]:
better_topo_data = json.load(open(topo_path))
l = better_topo_data['objects']['europe']['geometries']
better_topo_data['objects']['europe']['geometries'] = [d for d in l if d['properties']['NAME'] in better]

In [25]:
worse_topo_data = json.load(open(topo_path))
l = worse_topo_data['objects']['europe']['geometries']
worse_topo_data['objects']['europe']['geometries'] = [d for d in l if d['properties']['NAME'] in worse]

Building the cholorpleth.

In [26]:
m11 = folium.Map(location=[50, 10], zoom_start=4, tiles='cartodbpositron')

In [27]:
m11.choropleth(geo_data=better_topo_data, data=df.loc[df['diff']<0],
               columns=['code', 'diff'], topojson='objects.europe',
               key_on='feature.id',
               fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
               legend_name='Difference in unemployement rate with Switzerland (%)',
               threshold_scale=list(np.linspace(np.amin(df.loc[:, ['diff']].values), 0, 4)), name='unemployment layer')

In [28]:
m11.choropleth(geo_data=worse_topo_data, data=df.loc[df['diff']>0],
               columns=['code', 'diff'], topojson='objects.europe',
               key_on='feature.id',
               fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
               legend_name='Difference in unemployement rate with Switzerland (%)', name='unemployment layer')

In [29]:
folium.TopoJson(missing_topo_data, object_path='objects.europe', style_function=lambda feature: {
    'color' : 'black',
    'opacity' : 0,
    'weight' : 2,
    'dashArray' : '5, 5'
        }).add_to(m11)

Adding the markers.

In [30]:
m_c = MarkerCluster().add_to(m11)

for i in df.index:
    n = df.loc[i, 'country']
    p = df.loc[i, 'diff']
    if n == 'The former Yugoslav Republic of Macedonia':
        n = 'Macedonia'
    long, lat = centro.loc[centro.name == n, 'long'].values[0], centro.loc[centro.name == n, 'lat'].values[0]
    folium.Marker([lat, long], popup='{} : {}%'.format(n, p) , icon=folium.Icon(color='green')).add_to(m_c)

In [31]:
m11.save('maps/m11.html')
m11

Note that Norway is still in white as it has the same rate as Switzerland.

We note that most of countries in Europe have higher unemployement rate that swizerland.

Countries that are really different from Switzerland are darker than others (Norway is in white as noted above).

# 2) Suisse

Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

#### The assumptions

- When we import the data the unemployement rate that we get in the beginning is equals to the way the Swiss Confederation defines unemloyement
- We retrieve our data from the amstat website under '2.1 Arbeitlosenquoten' 
- We choose the german version of the data due to the fact that the topojson file has most of the country names in german. Therefore, we need the codes of these countries.
- Since Switzerland defines unemployment as the registered_jobseekers/active_population, we categorize the registered jobseekers into 2 groups : registered unemployed and registered non-unemployed.



#### Data loading and cleaning

In [32]:
#Read the data
df = pd.read_excel('data/unemployment.xlsx')

# drop useless columns
df = df.drop(['Gesamt','Gesamt.1','Gesamt.2','Gesamt.3'],axis = 1)

# drop first and last rows
df = df.drop(df.index[0])
df = df.drop(df.index[len(df)-1])

# rename the columns 
df.columns = ['canton','Rate','Registered unemployed','Registered jobseeker','Registered non-unemployed jobseeker']

df = df.reset_index(drop=True)

We convert the these respective columns which are of type string into numbers (float and int)

In [33]:
df['Rate'] = df['Rate'].astype(float)
df['Registered unemployed'] = df['Registered unemployed'].astype(int)
df['Registered jobseeker'] =  df['Registered jobseeker'].astype(int)
df['Registered non-unemployed jobseeker'] = df['Registered non-unemployed jobseeker'].astype(int)

In [34]:
#Cleaned data
df.head()

,canton,Rate,Registered unemployed,Registered jobseeker,Registered non-unemployed jobseeker
0,Aargau,2.9,10684,15145,4461
1,Appenzell Ausserrhoden,1.7,523,866,343
2,Appenzell Innerrhoden,0.7,62,102,40
3,Basel-Landschaft,2.8,4082,5540,1458
4,Basel-Stadt,3.5,3455,5168,1713


#### TopoJson setup

Now we want to attach the codes of each canton to our dataframe. Given a topojson file, we traverse this datastructure each
time retrieving the canton and its code.

However we must state that there are some incoherences with the names of cantons in the topojson file and our read data from amstat. 
We solve these incoherences by choosing a common name between the 2 datasets


In [35]:
# load topo json file
canton_geo_path = r'./topojson/ch-cantons.topojson.json'
with codecs.open(canton_geo_path, "r", encoding='utf-8', errors='ignore') as topo_data:
    topo_data = json.load(topo_data)

In [36]:
#we create a dictionary which has as a keys the cities and as values the codes 
canton_code = dict([(cantons['properties']['name'],cantons['id']) for cantons in topo_data['objects']['cantons']['geometries'] ])
    
#name incoherencies
old_code = ['Bern/Berne', 'Fribourg', 'Genève', 'Graubünden/Grigioni', 'Neuchâtel', 'Ticino', 'Valais/Wallis', 'Vaud']
new_code = ['Bern', 'Freiburg', 'Genf', 'Graubünden', 'Neuenburg', 'Tessin', 'Wallis', 'Waadt']

#we rename the old_code with the new code
for (old, new) in zip(old_code, new_code) :
    canton_code[new] = canton_code.pop(old)

We now finally combine the area codes with the our dataframe

In [37]:
codes = pd.Series(canton_code).reset_index().drop('index',axis = 1)
codes.columns= ['code']
df['code'] = codes

In [38]:
df.head()

,canton,Rate,Registered unemployed,Registered jobseeker,Registered non-unemployed jobseeker,code
0,Aargau,2.9,10684,15145,4461,AG
1,Appenzell Ausserrhoden,1.7,523,866,343,AR
2,Appenzell Innerrhoden,0.7,62,102,40,AI
3,Basel-Landschaft,2.8,4082,5540,1458,BL
4,Basel-Stadt,3.5,3455,5168,1713,BS


#### Display the data on a map

Let's build a chloropeth map displaying the unemployment rates of the cantons in Switzerland. Each canton will have a marker indicating its unemployement rate. 
The map is of course interactable

In [39]:
def makemap(topo_data, df, field, coordinates=[46.801111, 8.226667], legend='Unemployment rate (%)', t_min = None, t_max = None): 
    """
    This function is meant to build a cholorpleth.
    
    topo_data = data from the TopoJson file
    df = pandas DataFrame containing the rates
    field = field of df we which to display on the cholorpleth
    coordinates = in order to center the map
    legend = title of the legend
    """

    
    # get the range of the values
    if t_min is None and t_max is None:
        t_min = np.min(df[field].values.min())
        t_max = np.max(df[field].values.max())
    
    # intervals for the cholorpleth (by default we choose 6 intervals)
    t = list(np.linspace(t_min, t_max, 6))
     
    # create map
    m = folium.Map(location=coordinates, zoom_start=8, tiles='cartodbpositron')
    m.choropleth(geo_data=topo_data, data=df,
                 columns=['code', field], topojson='objects.cantons',
                 key_on='feature.id',
                 fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
                 legend_name=legend, threshold_scale = t, name='unemployment layer',highlight=True)

    # add markers
    m_c = MarkerCluster().add_to(m)
    for i in df.index:
        n = df.loc[i, 'canton']
        p = df.loc[i, field]
        long, lat = df.loc[df.canton == n, 'lon'].values[0], df.loc[df.canton == n, 'lat'].values[0]
        folium.Marker([lat, long], popup='{} : {}%'.format(n, p) , icon=folium.Icon(color='green')).add_to(m_c)
    return m

#### Obtain centroids for cantons
- Municipality data are obtained on the [OpenData](https://opendata.swiss/en/dataset/gemeindetypologie-are) platform, which provides a link to official data of the Confederation on commune boarders. They contain a field KT_KZ which is the code of the canton (Kanton-Kennzeichen). We need to dissolve this shapefile to obtain geometries for each canton.
- [Stackoverflow thread for dissolving geometries](https://gis.stackexchange.com/questions/149959/dissolving-polygons-based-on-attributes-with-python-shapely-fiona)
- Finally, we obtain the centroids by using the geopandas library, which allows getting the centroid of Shapefile geometries.

In [40]:
def dissolve_geometries():
    # define your directories and file names
    name_in = 'data/ARE_GemTyp00_9.shp'
    name_out = 'data/cantons_dissolved.shp'

    # create a dictionary
    states = {}
    # open your file with geopandas
    counties = GeoDataFrame.from_file(name_in, crs = {'init' :'epsg:21781'})
    for i in range(len(counties)):
        state_id = counties.at[i, 'KT_KZ']
        county_geometry = counties.at[i, 'geometry']
        # if the feature's state doesn't yet exist, create it and assign a list
        if state_id not in states:
            states[state_id] = []
        # append the feature to the list of features
        states[state_id].append(county_geometry)

    # create a geopandas geodataframe, with columns for state and geometry
    states_dissolved = GeoDataFrame(columns=['state', 'geometry'], crs=counties.crs)
    # iterate your dictionary
    for state, county_list in states.items():
        # create a geoseries from the list of features
        geometry = GeoSeries(county_list)
        # use unary_union to join them, thus returning polygon or multi-polygon
        geometry = geometry.unary_union
        # set your state and geometry values
        states_dissolved.set_value(state, 'state', state)
        states_dissolved.set_value(state, 'geometry', geometry)

    # save to file
    states_dissolved.to_file(name_out, driver="ESRI Shapefile")
    return states_dissolved

In [41]:
states_dissolved = dissolve_geometries()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


We now inverse the `canton_code` dictionnary to retrieve the names of cantons in the disolved centroids list.

In [42]:
#Reverse canton code dictionary previously defined
reversed_cantons_code = {v: k for k, v in canton_code.items()}

Now we extract the coordinates of the centroid of each canton.

In [43]:
# Each value of this dictionnary is a Point object. Attributes x and y are its coordinates.
gdf = states_dissolved.centroid.to_crs({'init': 'epsg:4326'})

# retrieving x values
x = np.asarray(gdf.centroid.map(lambda p: p.x))
# retrieving y values
y = np.asarray(gdf.centroid.map(lambda p: p.y))

# array of the canton codes
cantons_c = np.array(states_dissolved['state'])

#we build our centroid datastructure
centroids = []
for i in range(len(cantons_c)):
    centroids.append([x[i],y[i], cantons_c[i], reversed_cantons_code[cantons_c[i]]])
    
# build a DataFrame
canton_centroids = pd.DataFrame(centroids,columns=['lon', 'lat', 'code', 'canton'])
canton_centroids = canton_centroids.sort_values('canton').reset_index(drop=True)
canton_centroids.head()

,lon,lat,code,canton
0,8.156634,47.410525,AG,Aargau
1,9.368506,47.366144,AR,Appenzell Ausserrhoden
2,9.416338,47.317171,AI,Appenzell Innerrhoden
3,7.702226,47.451729,BL,Basel-Landschaft
4,7.615262,47.564895,BS,Basel-Stadt


#### Merging centroid coordinates in global data frame

In [44]:
df = pd.merge(df, canton_centroids)

In [45]:
df.head()

,canton,Rate,Registered unemployed,Registered jobseeker,Registered non-unemployed jobseeker,code,lon,lat
0,Aargau,2.9,10684,15145,4461,AG,8.156634,47.410525
1,Appenzell Ausserrhoden,1.7,523,866,343,AR,9.368506,47.366144
2,Appenzell Innerrhoden,0.7,62,102,40,AI,9.416338,47.317171
3,Basel-Landschaft,2.8,4082,5540,1458,BL,7.702226,47.451729
4,Basel-Stadt,3.5,3455,5168,1713,BS,7.615262,47.564895


### We plot the map

In [46]:
swiss_coordinates = [46.801111, 8.226667]

Here we display the chlorpeth map of the unemployments per swiss cantons

In [47]:
m2 = makemap(topo_data, df, 'Rate', swiss_coordinates, 'Swiss confederation unemployment rates')
m2.save('maps/m2.html')
m2

This map displays the Swiss confederation unemployment rate. As explained earlier, unemployment in switzerland is defined by the num
of registered job seekers divided by the active populuation. 

As we can see, this map doesn't distinguish between the people who are simply registered unemployed and the people who are registered non-unemployed
So we can see that these values are high compared to the the categorization we will show later on.

Here we derive formulas to calculate the rate of registered unemployed and the registered non-unemployed

- $i \in \{registered\_unemployed, registered\_non\_unemployed\}$
- $rate=\frac{N_{registered\_jobseeker}}{N_{active\_people}} \implies N_{active\_people}=\frac{N_{registered\_jobseeker}}{rate}$
- $  rate_i= \frac{N_i}{N_{active\_people}} = \frac{rate\times N_i}{N_{registered\_jobseeker}}$

This function takes the data frame in argument and returns another dataframe containing the rate computed using either registered unemployed or registered non unemployed (defined in argument `category`).

In [48]:
def stats_on_different_category(data, category):
    """
    Function that computes 
    """
    # compute rate corresponding to category (either registered unemployed and registered non_unemployed)
    data['Rate'] = (data[category] * data['Rate'])/data['Registered jobseeker']
    data['Rate'] = data['Rate'].round(1)
    # drop other columns
    data = data.drop(['Registered unemployed', 'Registered jobseeker','Registered non-unemployed jobseeker'], axis=1)
    
    # return data frame with just interresting values
    return data

#### Same scale

In ordre to build the maps of unemployement for both registered unemployed and non registered unemployed, we will use the same color scale. It will make in easier to compare.

In [49]:
registered_unemployed_rate = stats_on_different_category(df,'Registered unemployed')
register_non_unemployed_js = stats_on_different_category(df, 'Registered non-unemployed jobseeker')

In [50]:
t_min = np.min([registered_unemployed_rate.Rate.values.min(),register_non_unemployed_js.Rate.values.min()])
t_max = np.max([registered_unemployed_rate.Rate.values.max(),register_non_unemployed_js.Rate.values.max()])

#### Registered Unemployed category

In this section, we only take into account the people who are registered unemployed in the equation and we will attempt to see how our map changes. Since these values are the ones which represents the most unemployed.  

We plot the registered unemployed

In [51]:
m3 = makemap(topo_data, registered_unemployed_rate, 'Rate', swiss_coordinates, 'Rate for the unemployed (%)', 
        t_min=t_min, t_max=t_max)
m3.save('maps/m3.html')
m3

We can see that since these values seems to decrease due to us only taking into account the number of registered unemployed. Geneva still has the highest unemployement rate for this category with 4.1

#### Registered non unemployed
Finally we only take into account the number registered non unemployed. Basically these are people who might have an intermediate job and are seeking for a new one or the people who are jobless but haven't declared that they are unemployed.


In [52]:
m4 = makemap(topo_data, register_non_unemployed_js, 'Rate', swiss_coordinates, 'Rate for non-unemployed jobseekers (%)',
       t_min=t_min, t_max=t_max)
m4.save('maps/m4.html')
m4

Finally, the map shows that the unemployment rate is even less for these type of people. The maximum percentage is 0.4 and the minumum percent is 0.3.

The coloring of this map has changed with respect to the previous ones. For example, even though geneva might have the most unemployment rate, we would have thought that Geneva would lead the unemployment rates in all Categories. But this isn't true.

In fact, in this case, the Jura canton's unemployment rate is higher than Genevas.

# 3) Swiss and foreign workers

Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

#### Assumptions
- We download the website from the category "2.1 Arbeitslosenquote (worklessness rates)".
- We select data for September 2017, the month in which the latest SECO employment market report was released, to try to reproduce and validate their numbers as closely as possible.
- First, we have to do some cleanup, which is done below. The final dataframe will have a column per canton, two columns for worklessness rates in percentage per nationality (Swiss / foreigner), and two columns for the absolute number of registered worklessness people (Swiss / foreigners).

#### Importation of data and cleaning

In [53]:
worklessness_data = 'data/worklessness_ch.xlsx'

First, we import and preprocess the downloaded data.

In [54]:
df = pd.read_excel(worklessness_data)
df.head()

,Kanton,Nationalität,Monat,September 2017,September 2017.1,September 2017.2,Gesamt,Gesamt.1,Gesamt.2
0,NaN,NaN,Metriken,Arbeitslosenquote,Variationskoeffizient,Registrierte Arbeitslose,Arbeitslosenquote,Variationskoeffizient,Registrierte Arbeitslose
1,Zürich,Ausländer,NaN,5.3,A,12111,5.3,A,12111
2,Zürich,Schweizer,NaN,2.5,A,15114,2.5,A,15114
3,Bern,Ausländer,NaN,5.5,A,4900,5.5,A,4900
4,Bern,Schweizer,NaN,1.8,A,8758,1.8,A,8758


There are some data on canton, nationality, month (`NaN`), worklessness quote and number as well as a category, which we will not consider. First, we rearrange the data in a new dataframe, saving worklessness quotes and numbers for foreigners and Swiss people in separate columns.

Let's note that the total `Gesamt` is not differenciated according to the nationality.

In [55]:
# renaming the columns
df.columns = (['canton', 'nationality', 'month', 'unemployment_rate', 'd1', 'workless_registered', 'd2', 'd3', 'd4'])


# delete first row
df = df.drop(df.index[0]) 

# select interesting columns
df = df.drop(['month','d1','d2','d3','d4'],axis=1,)

# casting values into the right types
df.workless_registered = df.workless_registered.astype(int)
df.unemployment_rate = df.unemployment_rate.astype(float)

df.head()

,canton,nationality,unemployment_rate,workless_registered
1,Zürich,Ausländer,5.3,12111
2,Zürich,Schweizer,2.5,15114
3,Bern,Ausländer,5.5,4900
4,Bern,Schweizer,1.8,8758
5,Luzern,Ausländer,3.9,1593


In [56]:
# extracting unemployment rate for both Swiss citizens and foreigners
unemp_ch = np.asarray(df[df['nationality']=='Schweizer'].unemployment_rate)
unemp_for = np.asarray(df[df['nationality']=='Ausländer'].unemployment_rate)

# taking the difference in rates
diff_unemp_ch_for = unemp_for - unemp_ch

# extracting number of registered workless people for both Swiss citizens and foreigners
unemp_ch_reg = np.asarray(df[df['nationality'] == 'Schweizer'].workless_registered)
unemp_for_reg = np.asarray(df[df['nationality'] == 'Ausländer'].workless_registered)

# getting the list of cantons
cantons = list(np.unique(df[df.canton != 'Gesamt'].canton))

# put data in new, clean dataframe
df = pd.DataFrame({'canton':cantons,
                       'unemp_ch': unemp_ch,
                       'unemp_for': unemp_for,
                       'unemp_ch_reg': unemp_ch_reg,
                       'unemp_for_reg': unemp_for_reg,
                  'diff_unemp_ch_for': diff_unemp_ch_for})

df.tail()

,canton,diff_unemp_ch_for,unemp_ch,unemp_ch_reg,unemp_for,unemp_for_reg
21,Uri,2.4,3.5,8891,5.9,8264
22,Waadt,2.8,2.1,2690,4.9,2126
23,Wallis,3.2,4.2,2768,7.4,1970
24,Zug,0.9,4.8,6292,5.7,5942
25,Zürich,5.4,3.6,1114,9.0,505


The data above shows the relevant data extracted for September 2017: the name of the canton, the umemployment rate for Swiss people (`umemp_ch_reg`), foreigners (`unemp_for_reg`), as well as total number of registered workless people (`unemp_ch_reg` and `unemp_for_reg`). The difference between the foreign unemployed and Swiss workless quote per canton was derived and saved in `diff_unemp_ch_for`.

#### Checking the claim of the SECO on the difference between swiss people and foreigners.

In this data set we don't have national rates both for foreigners and Swiss people, we just have a total general rate. We are looking for the unemployement rates for foreign people and swiss people on national scale..

We cannot just take the mean of the rates for each canton as all the cantons are not populated the same way.

In order to compute unemployement rates for foreigners and Swissmen, we will compute the weighted mean (weights are based on the population of each canton).

This is first the unweighted mean of the rates. We will be able to compare it to the more precise approach.

In [57]:
df[['unemp_ch', 'unemp_for']].mean().round(2)

unemp_ch     1.96
unemp_for    4.47
dtype: float64

First we have to find the total number of unemployed people in Switzerland.



To find out the total number of unemployed people in Switzerland, we can ponder the percentage by the number of unemployed people (which gives us information on the total number of people living in the canton). We calculate the unemployment quote for Switzerland by calculating first the total Nulation:

$r_{unemployement,canton,swiss\_people}=\frac{N_{unemployed,canton,swiss\_people}}{N_{canton,swiss\_people}} \implies N_{canton,swiss\_people}=\frac{N_{unemployed,canton,swiss\_people}}{r_{unemployement,canton,swiss\_people}}$

$r_{unemplyement,canton,foreigners}=\frac{N_{unemployed,canton,foreigners}}{N_{canton,foreigners}} \implies N_{canton,foreigners}=\frac{N_{unemployed,canton,foreigners}}{r_{unemplyement,canton,foreigners}}$

Then, we compute nation rates of unemployement for swiss people and foreigners like in this formula:


$r_{unemployement, switzerland, Swiss\_people} = \frac{\sum N_{unemployed,canton,Swiss\_people}}{\sum N_{canton,Swiss\_people}}$

$r_{unemployement, switzerland, foreigners} = \frac{\sum N_{unemployed,canton,foreigners}}{\sum N_{canton,foreigners}}$

Get total populations of Swiss people and foreigners for each canton.

In [58]:
# total numbers of foreigners for each canton
df['pop_tot_for'] = np.round(df['unemp_for_reg'] / df['unemp_for']*100)
df['pop_tot_for'] = df['pop_tot_for'].astype(int)

# total numbers of Swiss people for each canton
df['pop_tot_ch'] = np.round(df['unemp_ch_reg']/df['unemp_ch']*100)
df['pop_tot_ch'] = df['pop_tot_ch'].astype(int)

In [59]:
df.head()

,canton,diff_unemp_ch_for,unemp_ch,unemp_ch_reg,unemp_for,unemp_for_reg,pop_tot_for,pop_tot_ch
0,Aargau,2.8,2.5,15114,5.3,12111,228509,604560
1,Appenzell Ausserrhoden,3.7,1.8,8758,5.5,4900,89091,486556
2,Appenzell Innerrhoden,2.6,1.3,2292,3.9,1593,40846,176308
3,Basel-Landschaft,1.7,0.4,59,2.1,53,2524,14750
4,Basel-Stadt,2.2,1.2,838,3.4,617,18147,69833


In [60]:
# get weighted worklessness rates
mean_unemp_ch = np.sum(df['unemp_ch_reg']) / np.sum(df['pop_tot_ch'])
mean_unemp_for = np.sum(df['unemp_for_reg']) / np.sum(df['pop_tot_for'])
print('unemployment rate for foreigners: ' + str(np.round(mean_unemp_for * 100,1))+ '%')
print('unemployment rate for Swiss people: ' + str(np.round(mean_unemp_ch * 100,1))+ '%')

unemployment rate for foreigners: 5.1%
unemployment rate for Swiss people: 2.2%


**Note**: We now get exactly the numbers that were mentioned in the report (`5.1%` for foreigners and `2.2%` for Swiss people)!

## Show Map of Switzerland
We first show a chloropleth map of Swiss and foreign worklessness rates for each canton. First, we show separate maps for foreigner and Swiss worklessness rates, then, we show the difference in worklessness rates between foreigners and Swiss.

We add the canton codes (such a BE for Bern) to the dataframe for later fusion with other datasources (latitude and longitude of centroids) and ease of use in mapping. We use the dictionnary defined in previous question.

In [61]:
df['code'] = codes

The code field is now added:

In [62]:
df.head()

,canton,diff_unemp_ch_for,unemp_ch,unemp_ch_reg,unemp_for,unemp_for_reg,pop_tot_for,pop_tot_ch,code
0,Aargau,2.8,2.5,15114,5.3,12111,228509,604560,AG
1,Appenzell Ausserrhoden,3.7,1.8,8758,5.5,4900,89091,486556,AR
2,Appenzell Innerrhoden,2.6,1.3,2292,3.9,1593,40846,176308,AI
3,Basel-Landschaft,1.7,0.4,59,2.1,53,2524,14750,BL
4,Basel-Stadt,2.2,1.2,838,3.4,617,18147,69833,BS


#### Centroids of cantons

We use again the centroid coordinates that we computed in the previous question.

In [63]:
df = pd.merge(canton_centroids, df)


Now that the centroid coordinates have been added, we can show a chloropleth map and a label at the centroids of each canton with the unemployment rates (unfortunately, Folium, doesn't offer too many visualization options, so printing labels on the areas is tedious and not very elegant...)


#### Same scale

In order to build the maps of unemployement for Swiss people and foreigners, we will use the same color scales (to be able to compare them more easily).

In [64]:
t_min = np.min([df.unemp_ch.values.min(),df.unemp_for.values.min()])
t_max = np.max([df.unemp_ch.values.max(),df.unemp_for.values.max()])

#### Swiss worklessness rates by canton

In [65]:
m5 = makemap(topo_data, df, field='unemp_ch', legend='Unemployement rate for swiss people (%)', 
            t_min=t_min, t_max=t_max)
m5.save('maps/m5.html')
m5

#### Foreigner worklessness rates by canton

In [66]:
m6 = makemap(topo_data, df, field = 'unemp_for', legend='Unemployement rate for foreigners (%)',
            t_min=t_min, t_max=t_max)
m6.save('maps/m6.html')
m6


The colors above (same scale) visually make it clear that worklessness is higher for foreigners than for Swiss people.

#### Difference between foreigner and Swiss worklessness rates
Finally, we can show the difference in rates between foreigners and Swiss people. A positive rate indicates higher foreigner worklessness rates, a negative value a higher Swiss worklessness rate.

In [67]:
m7 = makemap(topo_data, df, field = 'diff_unemp_ch_for')
m7.save('maps/m7.html')
m7

# 4) Bonus: 

*BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [Röstigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?

We can already see from the visualizations above that the French-speaking part of Switzerland has higher worklessness rates. To confirm this, we will calculate the mean worklessness rates in the cantons correponding to the language regions of Switzerland, similar as done before for foreigners / Swiss people. We will suppose that bilingual cantons belong to the region of the majority language group.